In [1]:
import praw
import pandas as pd
import unicodedata
from sqlalchemy import create_engine
from sqlalchemy.types import String

In [2]:
USERAGENT = '/u/insight_fu comments'
SUBREDDITS = ["pics", "politics", "leagueoflegends", "GirlGamers"]
DATABASE = 'recentPosts'
MAXPOSTS = 10

In [3]:
r = praw.Reddit(USERAGENT)

Version 2.1.21 of praw is outdated. Version 3.0.0 was released Tuesday June 16, 2015.


In [4]:
multi_reddits = []
for subreddit in SUBREDDITS:
    if subreddit == 'GirlGamers':
        multi_reddits.append(r.get_subreddit(subreddit).get_top_from_day(limit = MAXPOSTS))
    elif subreddit == 'politics':
        multi_reddits.append(r.get_subreddit(subreddit).get_top_from_day(limit = MAXPOSTS))
    elif subreddit != 'GirlGamers' AND subreddit != 'politics':
        multi_reddits.append(r.get_subreddit(subreddit).get_top_from_hour(limit = MAXPOSTS))

SyntaxError: invalid syntax (<ipython-input-4-af68d92c02b9>, line 7)

In [ ]:
for reddit in multi_reddits:
    print reddit

In [ ]:
subLinks = []
for reddit in multi_reddits:
    for post in reddit:
        subLinks.append({'subreddit': post.subreddit,
                        'postTitle': post.title,
                        'postURL': post.permalink,
                        'postCreated': post.created_utc})

In [ ]:
subLinksPD = pd.DataFrame(subLinks)

In [ ]:
subLinksPD.head()

In [ ]:
subLinksPD.iloc[0].map(type)

In [ ]:
convertThese = ['postTitle', 'postURL']
for convert in convertThese:
    subLinksPD[convert] = subLinksPD[convert].map(lambda x:unicodedata.normalize('NFKD', x).encode('ascii', 'ignore'))

In [ ]:
convertThese = ['subreddit', 'postCreated']
for convert in convertThese:
    subLinksPD[convert] = subLinksPD[convert].map(str)

In [ ]:
engine = create_engine("mysql+pymysql://root@localhost/" + str(DATABASE))
subLinksPD.to_sql('recent', engine, if_exists="replace")